<a href="https://colab.research.google.com/github/KorStats/skku/blob/main/code/MLM_%EC%A6%9D%EB%B6%84_%EC%B5%9C%EC%A2%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 35.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlink

In [ ]:
from transformers import pipeline, AutoTokenizer
import random

def mask_and_predict(sentence, model_name="klue/bert-base", num_masks=3):
    """
    주어진 문장에서 랜덤하게 num_masks 개의 단어를 마스킹하고, BERT 모델을 사용하여 복원하는 함수

    Args:
        sentence (str): 원본 문장
        model_name (str): 사용할 BERT 모델 (기본값: "klue/bert-base")
        num_masks (int): 마스킹할 단어 개수 (기본값: 3)

    Returns:
        tuple: (마스킹된 문장, 복원된 문장)
    """
    # 사전 학습된 BERT 모델 및 토크나이저 로드
    mlm_pipeline = pipeline("fill-mask", model=model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # 토큰화
    tokenized_ids = tokenizer.encode(sentence, add_special_tokens=True)
    tokenized_tokens = tokenizer.convert_ids_to_tokens(tokenized_ids)

    # 마스킹할 단어(어절) 선택 - [CLS]와 [SEP] 제외
    maskable_indices = [i for i in range(1, len(tokenized_tokens) - 1) if "##" not in tokenized_tokens[i]]

    # 마스킹할 개수 설정
    num_masks = min(num_masks, len(maskable_indices))
    mask_indices = random.sample(maskable_indices, num_masks)

    # 마스킹 적용
    for idx in mask_indices:
        tokenized_tokens[idx] = "[MASK]"

    # 마스킹된 문장 생성
    masked_sentence = tokenizer.convert_tokens_to_string(tokenized_tokens)

    # MLM 모델로 예측 수행
    predictions = mlm_pipeline(masked_sentence)

    # 여러 개의 [MASK]를 예측하여 문장 복원
    new_sentence = masked_sentence
    for mask_preds in predictions:
        best_pred = mask_preds[0]["token_str"]  # 가장 확률이 높은 예측값 선택
        new_sentence = new_sentence.replace("[MASK]", best_pred, 1)

    return new_sentence

# 데이터 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
'Colab Notebooks'  'My Drive'   이전   졸업논문


In [ ]:
import pandas as pd
df=pd.read_csv('/content/gdrive/MyDrive/졸업논문/DATA/NSMC_0.8_증분대상_1000개.csv', encoding='utf-8-sig')

In [ ]:
df.head(2)

,Unnamed: 0,id,document,label
0,0,7652286,솔직히 이영화때문에 슈퍼맨시리즈가 7년이란 세월동안 묻혀있엇다 맨오브스틸은 미국에서...,0
1,1,8895340,아무리 짱짱맨 옵대장의 다때려부수는 CG가 좋다그래도 이딴 디워랑 견줄만한 스토리로...,0


In [ ]:
df['MLM_aug'] = df['document'].apply(mask_and_predict)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu
Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpo

In [ ]:
df.head(5)

,Unnamed: 0,id,document,label,MLM_aug
0,0,7652286,솔직히 이영화때문에 슈퍼맨시리즈가 7년이란 세월동안 묻혀있엇다 맨오브스틸은 미국에서...,0,[CLS] ##에 이영화때문에 슈퍼맨시리즈가 7년이란 세월동안 묻혀있엇다 맨오브스틸...
1,1,8895340,아무리 짱짱맨 옵대장의 다때려부수는 CG가 좋다그래도 이딴 디워랑 견줄만한 스토리로...,0,[CLS] 아무리 짱짱맨 ##이대장의 다때려부수는 CG가 좋다그래도 이딴 인피니티워...
2,2,6549317,오프닝장면 죠스영화 고대루 베겼네?? 여자 비명 연기하는 것도 고대루네~! 그냥 갖...,0,[CLS] 오프닝장면 죠스영화 최고루 베겼네?? 여자 비명 연기하는 것도 고대루네 ...
3,3,8650550,대표적인 친일선전영화라서 최하점을 줄수밖에 없다. 영화수준 자체도 초딩 학예회수준의...,0,[CLS] 대표적인 친일선전영화라서 최하점을 줄수밖에 없다. 영화수준 자체도 초딩 ...
4,4,8658070,아~ 그렇구나~ 남편이 두 여자랑 바람펴도 조강지처 잘못인거구나~ㅎㅎ 부재는 '회개...,0,[CLS] 아 ~ 그렇구나 ~ 남편이 두 여자랑 바람펴도 조강지처 잘못인거구나 ~ ...


In [ ]:
df.to_csv('MLM증분.csv', encoding='utf-8-sig')